# Repeating a few useful functions from the previous weeks of this course.

* ReverseComplement(), finding the reverse complement of a gene.

* HammingDistance(), finding the hamming  distance between two genes.

In [ ]:
def ReverseComplement( P ):

  dictionairy = { 'A': 'T', 'G' : 'C', 'T' : 'A', 'C' : 'G' }

  P_r = P[::-1]

  P_rc = ""

  for i in range( len( P ) ):

    P_rc += dictionairy[ P_r[ i ] ]

  return P_rc



def HammingDistance( fg, sg ):


  distance = 0

  for i in range( len( fg ) ):


    if fg[ i ] != sg[ i ]:
      distance += 1

  return distance





# Finding all possible DNA strings of length k and then finding all k-mers of hamming distance at most d within the genome that are present.

In [ ]:

genomes = input("Enter genome:").split()

d = 1

k = 5

kmer = ""

freq_table = {}

new_freq_table = {}

alphabet = [ "A", "T", "C", "G" ]

dictionary = ["A" * k]

num = list( "0" * k )

len_alpha = len( alphabet )

g_length = len( genomes[ 0 ] )



def add( num, position ):

  if int( num[ position ] ) + 1 > len_alpha - 1:

    if position + 1 > k-1:
      return

    num[ position ] = '0'

    add( num, position + 1 )
    return

  num[ position ] = str( int( num[ position ] ) + 1  )

  kmer = []

  for i in range( len( num ) ):

    kmer.append( alphabet[int(num[i])] )

  dictionary.append( "".join(kmer) )
  

def find_kmers( genome ):
  for i in range( len_alpha ** k ):

    if int( num[ 0 ] ) + 1 > len_alpha - 1:

      num[ 0 ] = '0'

      add( num, 1 )
    
    else:

      num[ 0 ] = str( int( num[ 0 ] ) + 1  )

      kmer = []

      for i in range( len( num ) ):

        kmer.append( alphabet[int(num[i])] )

      dictionary.append( "".join(kmer) )



  for i in range( g_length - k + 1 ):


    # Add the k-mer if it is not in the frequency table.
    if genome[ i:(i + k) ] not in freq_table.keys():

      freq_table[ genome[ i:(i + k) ] ] = 1

    else:

      freq_table[ genome[ i:(i + k) ] ] += 1

  count = 0


  for kmer in dictionary:

    for key in freq_table.keys():

      if HammingDistance( kmer, key ) <= d:

        count += freq_table[ key ]

      #if HammingDistance( ReverseComplement( kmer ), key ) <= d:

        #count += freq_table[ key ]

    new_freq_table[ kmer ] = count

    count = 0


  buff = {}

  for x, y in new_freq_table.items():

    if y != 0:

      buff[ x ] = y

  return buff




new_freq_table = find_kmers( genomes[ 3 ] )

for kmer in new_freq_table.keys():

  kmer_present = 0

  for genes in genomes:

    #print( genes, kmer )

    for i in range( len( genes ) - k + 1 ):

      if HammingDistance( kmer, genes[ i:( i + k ) ] ) <= d:

        kmer_present += 1
        break



  if kmer_present >= len( genomes ) :

    print( kmer )




# Finding the profile matrix of a genome.

In [ ]:

kmer = list( input("Enter Genome: ") )

#kmer = list( "ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT" )

#profile = [['0.2', '0.2', '0.3', '0.2', '0.3'], ['0.4', '0.3', '0.1', '0.5', '0.1'], ['0.3', '0.3', '0.5', '0.2', '0.4'], ['0.1', '0.2', '0.1', '0.1', '0.2']]

profile = [ ]


for i in range(4):

  print( "Input The ", i, "th row of the profile matrix: ")

  profile.append( input().split() )


k = int( input("Enter k: ") )

#k = 5

dictionary = { "A": 0, "C": 1, "G": 2, "T": 3 }

prob_dict = {}

for i in range( len(kmer) - k + 1 ):


  prob = 1

  for j in range( k ):


    #print( (kmer[ i:(i + k) ][j]), profile[ dictionary[ (kmer[ i:(i + k) ][j]) ]  ], float( profile[ dictionary[ (kmer[ i:(i + k) ][j]) ]  ] [ j ] ))

    prob *= float( profile[ dictionary[ (kmer[ i:(i + k) ][j]) ]  ] [ j ] )


  prob_dict[ "".join( kmer[ i:(i + k) ] ) ] = prob


print( max( prob_dict, key=prob_dict.get ) )

## The Hamming distance function for reference again.

In [ ]:
def HammingDistanceArray( fg, sg ):


  distance = 0

  for i in range( len( fg ) ):

    for j in range( len(fg[0]) ):

      
      if list( fg[ i ] ) [ j ] != sg[ j ]:
        distance += 1

  return distance


# This code brute forces a motif from a genome.

* Initially selects a motif from the first k-length gene of each string.

* Scores this motif and then replaces each gene in each segment of the motif to get a better scoring motif.

* Outputs the overall best motif from it's search.

In [ ]:


DNA = input("DNA strings: ").split()

k = int( input("k: ") )

initial_motif_matrix = [ string[0:k] for string in DNA ]

len_DNA = len(DNA)

alphabet = [ "A", "C", "G", "T"]

alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }

motif = []

best_motif = []

motif_score = 0

best_ham = 0



def Score_Motif( motifs, consensus_string ):

  score_motif = 0

  for kmer in motif:

    for i in range( k ):

      if kmer[i] != consensus_string[i]:

        score_motif += 1

  return score_motif



def Find_Profile( motifs, position ):


  for i in range( len_DNA - position ):

    freq_table = {}

    for j in range( 1, len( DNA[0] ) - k + 1):

      motif_i_trial = DNA[ position + i ][ j:(j + k) ]

      freq_table[ motif_i_trial ] = HammingDistanceArray( motif[0:position], motif_i_trial )

    motif[ i + position ] = min( freq_table, key=freq_table.get )


def Find_Profile2( motifs, position ):


  # Generate a blank profile matrix.
  profile = [ ]

  consensus_string = ""

  consensus_prob = []

  for nuc in alphabet:

    blank_kmer = [ 0 for i in range( k ) ]

    for mot in motifs[ 0:position ]:

      for j in range( k ):

        if mot[ j ] == nuc:

          blank_kmer[ j ] += 1


    profile.append( blank_kmer )


  for i in range( k ):

    total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

    for j in range( 4 ):

      profile[ j ][ i ] = profile[ j ][ i ] / total_freq

      consensus_prob.append( profile[ j ][ i ] )

    consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
    consensus_prob = []
  

  # Need to check whether this piece of code works.
  for i in range( len_DNA - position ):

    freq_table = {}

    for j in range( 0, len( DNA[0] ) - k + 1):

      motif_i_trial = DNA[ position + i ][ j:(j + k) ]

      prob = 1

      for p in range( k ):

        # There may be an error here. 
        prob *= profile[ alpha_dict[ motif_i_trial[ p ] ]  ] [ p ]

      freq_table[ motif_i_trial ] = 1 - prob

    motif[ i + position ] = min( freq_table, key=freq_table.get )


  return consensus_string



# Crafting the initial motif matrix.
motif = initial_motif_matrix

#best_ham = HammingDistanceArray( motif[ 0:len_DNA ], motif[ 0 ] )


profile = [ ]

consensus_string = ""

consensus_prob = []

for nuc in alphabet:

  blank_kmer = [ 0 for i in range( k ) ]

  for mot in motif:

    for j in range( k ):

      if mot[ j ] == nuc:

        blank_kmer[ j ] += 1


  profile.append( blank_kmer )


for i in range( k ):

  total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

  #print( 'Profile matrix before modification: ', profile )

  for j in range( 4 ):

    profile[ j ][ i ] = profile[ j ][ i ] / total_freq

    consensus_prob.append( profile[ j ][ i ] )

  #print( 'Profile matrix after modification: ', profile )

  #consensus_string += list(alpha_dict.keys())[list(alpha_dict.values()).index( max( consensus_prob ) )]

  #consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
  consensus_string += alphabet[consensus_prob.index( max( consensus_prob ) )]
  consensus_prob = []

  #print( consensus_string )

print( 'profile matrix: ', profile )

best_score = Score_Motif( motif, consensus_string )

print( "Initial motif: ", motif )


for j in range( 1, (len( DNA[0] ) - k + 1) ):

  motif[ 0 ] = DNA[ 0 ][ j:(j + k) ]

  #print( motif[ 0 ] )

  #print( DNA[0], DNA[ 0 ][ j:(j + k) ] )

  for i in range( 1, len_DNA ):

    # I don't understand whats going on here.
    consensus_string = Find_Profile2( motif, i )
    
  Motif_score = Score_Motif( motif, consensus_string )
  #print( "\n", motif, " ", Motif_score, " ", consensus_string )

  if best_score > Motif_score:

    best_motif = motif.copy()
    best_score = Motif_score

  #print( "\n Motif: ", motif )

  #print( "\n   Best motif: ", best_motif )

print( best_motif )





In [ ]:
# An test input to the previous function.
GGCGTTCAGGCA AAGAATCAGTCA CAAGGAGTTCGC CACGTCAATCAC CAATAATATTCG
3

In [ ]:
# Outputting the results of the previous algorithm, but separated by newlines
# as opposed to spaces.
for string in best_motif:

  print( string )

# Making modifications to the previous block as I found some bugs.

In [ ]:
DNA = input("DNA strings: ").split()

k = int( input("k: ") )

initial_motif_matrix = [ string[0:k] for string in DNA ]

len_DNA = len(DNA)

alphabet = [ "A", "C", "G", "T"]

alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }

motif = []

best_motif = []

motif_score = 0

best_ham = 0



def Score_Motif( motifs, consensus_string ):

  score_motif = 0

  for kmer in motif:

    for i in range( k ):

      if kmer[i] != consensus_string[i]:

        score_motif += 1

  return score_motif



def Find_Profile( motifs, position ):


  for i in range( len_DNA - position ):

    freq_table = {}

    for j in range( 1, len( DNA[0] ) - k + 1):

      motif_i_trial = DNA[ position + i ][ j:(j + k) ]

      freq_table[ motif_i_trial ] = HammingDistanceArray( motif[0:position], motif_i_trial )

    motif[ i + position ] = min( freq_table, key=freq_table.get )


def Find_Profile2( motifs, position ):


  # Generate a blank profile matrix.
  profile = [ ]

  consensus_string = ""

  consensus_prob = []

  for nuc in alphabet:

    blank_kmer = [ 1 for i in range( k ) ]

    for mot in motifs[ 0:position ]:

      for j in range( k ):

        if mot[ j ] == nuc:

          blank_kmer[ j ] += 1

    profile.append( blank_kmer )


  for i in range( k ):

    total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

    for j in range( 4 ):

      profile[ j ][ i ] = profile[ j ][ i ] / total_freq

      consensus_prob.append( profile[ j ][ i ] )

    consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
    consensus_prob = []
  

  # Need to check whether this piece of code works.
  for i in range( len_DNA - position ):

    freq_table = {}

    for j in range( 0, len( DNA[0] ) - k + 1):

      motif_i_trial = DNA[ position + i ][ j:(j + k) ]

      prob = 1

      for p in range( k ):

        # There may be an error here. 
        prob *= profile[ alpha_dict[ motif_i_trial[ p ] ]  ] [ p ]

      freq_table[ motif_i_trial ] = 1 - prob

    motif[ i + position ] = min( freq_table, key=freq_table.get )


  return consensus_string



# Crafting the initial motif matrix.
motif = initial_motif_matrix

#best_ham = HammingDistanceArray( motif[ 0:len_DNA ], motif[ 0 ] )


profile = [ ]

consensus_string = ""

consensus_prob = []

for nuc in alphabet:

  blank_kmer = [ 1 for i in range( k ) ]

  for mot in motif:

    for j in range( k ):

      if mot[ j ] == nuc:

        blank_kmer[ j ] += 1

  profile.append( blank_kmer )


for i in range( k ):

  total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

  #print( 'Profile matrix before modification: ', profile )

  for j in range( 4 ):

    profile[ j ][ i ] = profile[ j ][ i ] / total_freq

    consensus_prob.append( profile[ j ][ i ] )

  #print( 'Profile matrix after modification: ', profile )

  #consensus_string += list(alpha_dict.keys())[list(alpha_dict.values()).index( max( consensus_prob ) )]

  #consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
  consensus_string += alphabet[consensus_prob.index( max( consensus_prob ) )]
  consensus_prob = []

  #print( consensus_string )

print( 'profile matrix: ', profile )

best_score = Score_Motif( motif, consensus_string )

print( "Initial motif: ", motif )


for j in range( 1, (len( DNA[0] ) - k + 1) ):

  motif[ 0 ] = DNA[ 0 ][ j:(j + k) ]

  #print( motif[ 0 ] )

  #print( DNA[0], DNA[ 0 ][ j:(j + k) ] )

  for i in range( 1, len_DNA ):

    # I don't understand whats going on here.
    consensus_string = Find_Profile2( motif, i )
    
  Motif_score = Score_Motif( motif, consensus_string )
  #print( "\n", motif, " ", Motif_score, " ", consensus_string )

  if best_score > Motif_score:

    best_motif = motif.copy()
    best_score = Motif_score

  #print( "\n Motif: ", motif )

  #print( "\n   Best motif: ", best_motif )

print( best_motif )

In [ ]:
# Again outputting the best motif separated by newlines as opposed to spaces.
for string in best_motif:

  print( string )

# An extension of the Hammign distance function.

In [ ]:
def HammingDistanceArray( fg, sg ):


  distance = 0

  for i in range( len( fg ) ):

    if fg[ i ] != sg[ i ]:
      distance += 1

  return distance

In [ ]:

def DistanceBetweenPatternAndStrings(Pattern, Dna):

  Dna = Dna.split()

  k = len( Pattern )
  distance_string = 0
  distance = 0

  for string in Dna:

    #print( string )

    distance_string = HammingDistanceArray( Pattern, string[0:k] )
  
    #print( 'distance_string initial: ', distance_string )

    #print( len(string) - k + 1 )

    for i in range( len( string ) - k + 1 ):
      

      #print( string[ i:(i + k)] )

      #print( 'AAA', string[ i:(i + k)], HammingDistanceArray( Pattern, string[ i:(i + k)] ) )

      if distance_string > HammingDistanceArray( Pattern, string[ i:(i + k)] ):

        distance_string = HammingDistanceArray( Pattern, string[ i:(i + k)] )


    #print( 'distance_string then:', distance_string )

    distance += distance_string

  print( distance )




In [ ]:
DistanceBetweenPatternAndStrings( 'GAGTACA', 'CCAGCGCTGCTTTATTGTTAAGCCGTTATCCTCCAAGTTGAGTCCTCGCGTAGCTCTAGAACGTCGGATCGGCCCACAACCAACAAACTAAAAATCAGGCGAGATTGCA TGAGTGGACAAACGGTTATGAAATCTGTATCCTCAATCTTAGGTGGACTCTCGCGCAGAGCGCGTGTACGGAGAGCTGGTCCCGGTGACCAAAGATCGTTCAGATTCGT GAGAGCGTTGCCGAGTAGCCCCATACCAACATGGTTCCACGGATCGTTTATCCCCTCGGACCTGGTTCCTGTCCACGCTATTTCCAAACGCTAGATATTTCGAGTCTAG ATTCTCAACGCAGTCGGCCCAAGTGCGGTCATACGGCATGAAGTATCATCCAATAAAAGTAAACGCCTCATATATCCATAGAGTACACAATCCGAGTAAATACCAGTCC CTTCCCGTCCACATCCTGTACCTGAGGCACTGGGCTGCCATAGATAAAAAAAGGGAACTAACGATTAGACCGTTGCCTGGATGGCCGGACGTCTCTTATCCCGCCACAT TCGGAAGTTTAAGGCAACGTCTTCGCGACGATAACGCTCACAACAGGCGCGCACCACAGGCCGGACTGGTATATGACAGGCTTCGGATGGATGATGTCGATATCGTATG ACGTAGTTCGATATATCACCTCGACCCTATGCGGTCTTGAGATCAGCAGGCTATGGCGATCAACAAACGCGGAAGCACCGGGGTACAGCCGGACTTGGTCTGCTAGATC TCGGGACTCATCTCAGCTAATACGCGATCTTGGTTGTGGTTTATCGTTGGGAGCAGAGCCCCCCGAGTTCAAGGTTCTGGGTTAGATACGATTCTCCCGCCAGGTCGTG AATTAAATTGGGCCCGAAGCTAAGATTTTCCGGGGAGCGCCGGTCCAGGATCTGTGACTCAAAACATACAACCCCAGTCAGCCTTAGCCGAAGTGGTCTCCCTCCGGGC TCGAAGAGAAGCCTCATCATCAGCGACCGTTCCTAATTTAGTAGGAAATTCGCAAATTGCGACGCAGTATAACCTCTAAGCCTGGACCTTCGATGCGGAAACGAAAGGG CGGCATTGGCGTTTACTATTATTTCTTTTAGCTTCGCTTTAATTCTGGTGTAACATTCCCGATACGCGATCTCATTCCTGTCGCGGTGGAGTCCTGGTACGCAATGATC CTTGGTGAATCGTGCCTCTAGCGGCAGTGGTTCGACGCTTCGGCAAACTTCAGGCCGTTCTAGTACGCTCCTTCGTAACTGCGACCACCCCCTTGATGCGAGAGCTGTT CTATTGACGAGCGATGGGAGCACATAAAAGAGGTTTGGTATATGCCCGCCGATTAGCACCTTAGGGCAGAGGACGTTGGGCAAAATTCTGGCATCTAAGTGTTAGGCTA AAAGAAAGTAAGGCCCGAGACGACGTTACGTCAGCGTAACGGCCATACGCTAGATGTCATTAAACAGTATTGTACCCGACGGTTTATCTTGGCTCATACGCTTTTAACG ATTATCCTCATAGTAATTCAATGCATTCATCCACTGATAGAAAATTATAACCTCCCTATGGAGCCCTTCCGGCACCCTTTGGGCTATTGCGAGACTGTCCGCTAGTAAC GTTTTCGATTGCAGGCTGTGCGTCCTTAGCGCGGGCAGTTAATATAAAGGGATTTGTTTCCTACCTCTTATATGCCGATAGCACAGATCAACGAATGAAAGACCAAATT AGGGGTCGCCTCTTACCTGCGCCCCTCTGGAGCCCTTTATAGACTTTCTAAGCAAATGCTGATAACTCGTGTGCGTAGAGTTCTGAATAGGCAAACAAGGTACTATAAG CCGAACTCCATCCAAACAAAATTGCTTCACGTCGCAGGATAGATGACGCAAATGAGGCAACTTGTTCGTATAACGGTTGTCGTACGCATCATTCGCCACATAAAGTACA AACCTGACCTCAGAGTTCCGCCATGTTTTTCCCCGAACCAGCGACATCGCATCCTGTCTTTTCGCGGGAAATGCCAACTAGAGCGGCATGACTGAGGGCAGTGGATCGG GCCATAGTGTCCATATATGACACAGGTCGTTTTTTGGAGTTTGTCGGCAACAAAGGACTGATTATCTGTACAGTTAAACGCCCCATCGATCGAGGGCCTGTAATTTTCT GCACATGATCCACGGATCAATTGTAACTGGGGTAAAGAAGCTGCAGGTCAAAAGGTTCGCTTGACTAAATGCCGGATAGCGCCAAGCCAGGCTCTCTAGGGTTCTTCAC CAGCGGTACGGAAACGCCTCAAGCGGGTGGATATGGCCGCGGCCACACCAACCAGGGGCGCCCATGAAGCGGAAGGCTTTCTATCTAGCATTCTGGAATAGTCCATATG TGATTCATTCCCGAATACTCCAAGTGGGAGAATGGTCCTCCTGGGGTGTGTACTACCGCGTGAATAAGTACTTCCGAGTAATACACCTCCGAATTTTGATATTAATGGA TGTGGTGGTAACGTAAAACGGATAAAGCATGCACACACTAACGGCCTAGTCTGGATTCCACGCCCAGGAATCTTCAGGCACATTAGGATGATGCCTTTTAATATCTACG TCGGAGATGGAAGCTACAGGTCTATTAGACCCTGTAAGGTCGCGTGCCCCATCGCAAATAACCGCGCAGGCTGCCCACGCCATTGTAGTGTTTCTGCGCGCCAGCAGAT GATGCTCCCACTGGTCCAAAAAAACTGTAGTGAATACCCCCCTGACACCGGCGCCTGGACATGATTCGGCGACGGTTGCTCTGCTGCCTATGAGTTTCCTGCGAATACG CGTAATCGGAAAGTGCGTCCAGAAGCCTCTGCGGGCTCTACTGTCGCAGTCATGTCTCAAAGAAAAGACGCATTCCAAACCATTCAATCTGACACGTCATCACTTACGT ACACTCCCATATGAAGAGTCGTGCCGCACTAGGACGCGTGTGACAAACCCTTGGTAAGCTGGCAGAGGAGAACCATTAGTCTTGAGCGGTTTTCCCGAACCATCTAGGT TCACCGTTAATGCAGCGCCGATCGAGTTTGGACCACTCGTCTTGACACCTTCATCGTTACGTGCGACATGGAACGCTAGCGGCGTAAAGAGGTAGTTGGGACGGTGTGT AAAATTGGTTAAATACGATCAGGCTACAGCCCTGGGCGGCCGCATCCATATGACTGCTCCCATACTATAAGTCGATTACAGGGCACACGGTTTATGAATTTGCAGTTGC CGGGAAGGCTAAGTGAAAGTACCTGCCCTGCAGCGTGTTACTAGTCTGGCACACTGGGTTTGCCGCGCTACATGCCATGGAGTGATCCCTGAGCCACCAATGTCGGCCA TATGCCCGGAGTCATAGTAGGTGAGGCCTGTGTCGGACTCACTCGGGTCCATTCACGTACAACTCGGACGTTGATAGAAAAGCCAATGCAGCGTGACCAGGCAATGACG GCTTAATGAGAGCTCAGCGATGCCTCGTCTAGAACCGCATATCCGTGCCAAACAATCAATTAGTTTCGTAACAGTCACCTCACTATTGCCTATTGGTCAGATGAGTCGA CGGGGCCAAGGAAGGTATGCGTCGGATTACAAAATCCCTGGAACGGGACTCTCTAAATACCTGGACGGGGACCCTACATTAAATATTAAAATAGACTATACCTACTGAA AGAAATGAATCAAGGCTCGGTGTGTTACGCGTCGATTGCAACGTCCCTTAAGCCTTTACGTACCGGGGGAGGTTTGGGGTAGTGCGACTTAGGGCTATGCACGACTCCA CAATTCTTAATGTGCGGATAAGACTGTTAAACATGCCGTTGGAATCTGGAAGGCTCCATCGGCTCCCTGCCTACCTAGCAAACCTCTAATTATCGGAACTGTTTGAGAC AGTAGCGCGCCTGAACTCAAAGGAAGTGAACACACGTAGCACGAGAACGAGAGATGTTAAGACAAGCCCGTAACAGACAATATACCGTGGAATTATTATGGGGCCCCTA GCCGCTAACTTAGCTTGTTTGAAAAGTCTGCCCACCGTAATGGAGGGCCCCACTCAAAGTGAAAGAAGAGGTCCCTCGCGGCTACTCTACCTCGAGCCCCGAAGAAGCC TATTACGCATGGTCGCCACCAAGACAACAGGGTCTTATAGGGGCACGCTAAAAGAAGTATTATCACATCGAAACAATCACGTCAACCCGCTCCGAACGGTGGAAAACAG CTCAACATTGACTATATCTGAACAAGACTACTGATACGCGTCAGTGTCCAAGCTGTGGAAGCAATCACACATGTGGCCCGCAACCCAAGTTTTAAGAATTCTCGTTTCA' )

In [1]:
# Testing my sanity.
print( 'TTACCTTAAC GATATCTGTC ACGGCGTTCG CCCTAAAGAG CGTCAGAGGT'.split() )

# A motif scoring function.

In [ ]:
def Score_Motif( motifs, consensus_string ):

  score_motif = 0

  for kmer in motifs:

    for i in range( len( kmer ) ):

      if kmer[i] != consensus_string[i]:

        score_motif += 1

  return score_motif

For this challenge I need to generate a random motif, then find the profile and then find a motif from the profile. Continue this until the score of the motif no longer improves. 

I do the above 1,000 times and then choose the one with the lowest score.

In [ ]:
import random


alphabet = [ "A", "C", "G", "T"]
alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }
motif_score = 0
best_score = 0


def randomizedMotifSearch( k, n, DNA ):

  DNA = DNA.split()
  genome_length = len( DNA )
  motif = []
  best_motif = []
  kmer = ""
  first_motif = True
  print( 'This is running ')

  for i in range( genome_length ):

    # Selecting the DNA string to work on.
    string = DNA[ i ]
    # Determining the kmer being extracted from the DNA string.
    rand_num = random.randint( 0, len( string ) - k  )

    # Appending the sleected kmer to the motif array.
    motif.append( string[ rand_num:(rand_num + k) ] )


  # n is the number of iterations.
  for j in range( n ):

    #print( motif )

    # Generate a blank profile matrix.
    profile = [ ]

    consensus_string = ""

    consensus_prob = []

    for nuc in alphabet:

      blank_kmer = [ 1 for i in range( k ) ]

      for mot in motif:

        #print( 'mot: ', mot, 'motif: ', motif, 'k: ', k )

        #print( 'motif: ', motif )

        for j in range( k ):

          #print( mot, j )
          if list( mot )[ j ] == nuc:
            

            blank_kmer[ j ] += 1

      profile.append( blank_kmer )


    for i in range( k ):

      total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

      for j in range( 4 ):

        profile[ j ][ i ] = profile[ j ][ i ] / total_freq

        consensus_prob.append( profile[ j ][ i ] )

      consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
      consensus_prob = []


    #print( 'profile: ', profile )

    motif_score = Score_Motif( motif, consensus_string )

    if first_motif == True:
      best_score = motif_score
      first_motif = False


    elif motif_score < best_score:

      best_motif = motif.copy()

    for l in range( len( DNA ) ):
      
      for m in range( 0, len( DNA[ l ] ) - k + 1 ):

        freq_table = {}

        kmer = DNA[ l ] [ m:(m + k) ]

        prob = 1

        for p in range( k ):

          # There may be an error here.
          prob *= profile[ alpha_dict[ kmer[ p ] ]  ] [ p ]

        freq_table[ kmer ] = 1 - prob

      motif[ l ] = min( freq_table, key=freq_table.get )

    #print( motif )

  # Outputting the final resulting motif.
  for kmer in best_motif:
    #print('This is also running')
    print( kmer )

  return best_motif


In [ ]:
best_motif = randomizedMotifSearch( 8, 1000, 'CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG TAGTACCGAGACCGAAAGAAGTATACAGGCGT TAGATCAAGTTTCAGGTGCACGTCGGTGAACC AATCCACCAGCTCCACGTGCAATGTTGGCCTA')

In [ ]:
print( best_motif )

# A variant of the randomized motif searching algorithm.

In [ ]:

import random



alphabet = [ "A", "C", "G", "T"]
alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }
motif_score = 0
best_score = 0
overall_best_score = 0
overall_best_motif = []

random.seed(0)

def randomizedMotifSearch( k, n, DNA ):
    
  random.seed(0)
  
  alphabet = [ "A", "C", "G", "T"]
  alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }
  motif_score = 0
  best_score = 0
  #overall_best_score = 0
  overall_best_motif = []

  DNA = DNA.split()
  genome_length = len( DNA )
  motif = []
  best_motif = []
  kmer = ""
  first_motif = True
  first_run = True

  for i in range( n ):

    motif = []

    # Randomly generating a motif for the initial starting motif.
    for j in range( genome_length ):

      # Selecting the DNA string to work on.
      string = DNA[ j ]
      # Determining the kmer being extracted from the DNA string.
      rand_num = random.randint( 0, len( string ) - k  )

      # Appending the sleected kmer to the motif array.
      motif.append( string[ rand_num:(rand_num + k) ] )

    first_motif = True

    while True:

      # Generate a blank profile matrix.
      profile = [ ]

      consensus_string = ""

      consensus_prob = []

      for nuc in alphabet:

        blank_kmer = [ 1 for i in range( k ) ]

        for mot in motif:

          #print( 'mot: ', mot, 'motif: ', motif, 'k: ', k )

          #print( 'motif: ', motif )

          for j in range( k ):

            #print( mot, j )
            if list( mot )[ j ] == nuc:

              blank_kmer[ j ] += 1

        profile.append( blank_kmer )


      for i in range( k ):

        total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

        for j in range( 4 ):

          profile[ j ][ i ] = profile[ j ][ i ] / total_freq

          consensus_prob.append( profile[ j ][ i ] )

        consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
        consensus_prob = []


      #print( 'profile: ', profile )

      motif_score = Score_Motif( motif, consensus_string )

      if first_run == True:
        overall_best_score = motif_score
        first_run = False

      if first_motif == True:
        best_score = motif_score
        first_motif = False

      elif motif_score < best_score:

        best_motif = motif.copy()
        best_score = motif_score

      else:
        
        break

      #print( overall_best_score, motif_score )

      if motif_score < overall_best_score:

        overall_best_score = motif_score
        overall_best_motif = motif.copy()

      #print( profile, motif )

      for l in range( len( DNA ) ):
        
        freq_table = {}


        for m in range( 0, len( DNA[ l ] ) - k + 1 ):

          kmer = DNA[ l ] [ m:(m + k) ]
          #print( kmer )

          prob = 1

          for p in range( k ):

            # There may be an error here.
            prob *= profile[ alpha_dict[ kmer[ p ] ]  ] [ p ]

          freq_table[ kmer ] = 1 - prob


        motif[ l ] = min( freq_table, key=freq_table.get )
        #print( motif )

    


  # Outputting the final resulting motif.
  blank = ""

  for kmer in overall_best_motif:
    #print('This is also running')
    #print( kmer )

    blank += " " + kmer

  print(blank)


  return overall_best_motif

In [ ]:
random.seed(0)
best_motif = randomizedMotifSearch( 15, 1000, 'CGAAATCTAAAACAAGGCGGAAGGAAATCAAAGGCCAACCCTTCTCGGATACAAGTTAGTGAAGTCGGGGCATGTTGGCGGTAGGCGTAACACGCGGGGTCTCGCTGTTGGAGGGAAAACTCGTATTACTGGTTACGACCCTCCTAAATCAATGTCACCGGCCGGTCACCGGGAACAGCGCCTGCGAAATCTAAAACAA GGCGGAAGGAAATCAAAGGCCAACCCTTCTCGGATACAAGTTAGTGAAGTCGGGGCATGTTGGCGGTAGGCGTAACACGCGGGGTCTCGCTGTTGGAGGGAAAACTCGTATTACTGGTTACGACCCTCCTAAATCAATGTCACCGGCCGGTCTATTCACGTGCTTCAACCGGGAACAGCGCCTGCGAAATCTAAAACAA ACATCCGAGCGTGCTTCATGCGTCTTTCGCTCAGGACAGAGGATCTTGGCTCAGGCGCTGACCCCGCTCTCTTTTGCTGGGCACAGAGCGGTTCGGGATCAGTGCTCCGTTGTATATTACGACAACATCTAACTTTGTACATAGCGCGGGCTGAAGGTGTGACATCCCAAGACGCGCGTACTTGGTCATCGATATGTGG ATTGAATGCTCCACATCTAGAGGGACGCGAGTAACAATTCGAGTTAGAAGGAGAATTCCCCTGTCTCACATCAGGCTTCACTTCGCGCATTGTTAATCGAGCCGATTAAGCGTGCTTCCAGTTATAGGTCACTAACCCGGGCACCCATCCGTTCTTAGGGGACACCTCTGTAGACAACAAGCGTCATGGACGCTAACTA TAAGAGGGAGGCTCCTCGCAAGAAGTGAAATATTCAGGCGGCTGATGACAGCCGCGCGAACGTCTTTCTGTATCGTGATTAGGTAACGGCCAGTTGTCCGACGTACCGCCTCCCATTCTACAGGTTGCCGGGAAGTGTAAAGCACCTTAATATGACGTTAATATGGGTGACTCCGGAGGATATATCGGTGCTTCATCCG CAATGGAAAGCACCCACACAAAGTAATATAAGCGTGGAGCAAGCGGAGTGTATACTTTTTAAACCGTAAATCAAATAGACTACGCAGTCGTAATAGGTAAACGGAGGCGTGAAAGCAGATTCCAAATCAACCACGCGGGTTCTCTCTGATAGCAGTTAGTAGTGCTCCCCAATTTGCTAATTTCCGTAAGAGCTGTTAA AATCATCACTAGTAGATTAGAACGAGGAGTTATAACTTTGCTTCACATTCATTAAAATTGTGAAGGATGTTACTCGTCGGTCCCCCTCTCGACTTCTCAGAATAAGTGACTCCTGGATGACCAGATTACATGTTCACGTCGCGTAACACCGTTCTGCGGCATACGTGGCGTTCCGACAGTCGGTTGAATGGTCCATGCG AGAAAGCTGCTCCAATTACCTGATTAGCCGTCGATGGGATCCCGGAAGCTACCGAGCCACACTGGGTTTGTAACAGGTCAGCAAGACAAGTCAATTACCCAGGTCGAGCTGAATCGATATCCCCGTGCTTCAGGGAGACCAATGGGGACGTTAGGACCACGCGACCCGAAGGAAAGGGATACCGTTGGATATGCTAGCT CCACATGCTAACGCAAAATAGCCACGAGGCCGTAGGATGGAGTTTCCTAGAGACCTTGGTAGCACCTCGTCATTAGTTGACAATCTTAGGTAGCAGTATCATCAGCCGTGAGCGGGGGCTGCGCGTTCCACTTTTAGACTTGTACATCCGAAAGCAAAGCGTGCTTCATGGACCTGATAGCGCGTCAGTAGTGTTCTGG TTTCACATGACTCCATCAGTTCTGGAGCTGGTGGAGTACTGGTTCGTGGCGAGCTTGTCAGTCGCGTCACAAGTATAAGCGTGCAAAATTCTCCACTGCGCTGTAGAGTAAACAGAAGGCCGTATTAAACAGAAGTAATTGAGAAGATGTGGCTCTTGGAGAGACAACCGCTCTTAGCGACCAGTCTAGCGAATGAGAT GCGACAATCACCCAAAACGTAACCTAACCGTCAAACGTTATAAGCCGCCTTCAGACTAACGGCTAAGTTGGGCCAGTACCAGCTTAAAGTCGACCGTCACCATTCCAATGCACATCTTTTCGCTCAAGCGACAGGTCTGTCGAGCTGCATGTGGCTTTTGTGCTCGTATCAGAACGTCGGGCCGACTCGGCTTTGTGAC TAGTTTGCATTTATATCCTGGCACACGCTGTTCAGGCTGTAAAATTTGATTGGTTGTCAGCATACTGTCCGTGCATTAACGGACTGTTTATACCGGCGTGCTTCATTGTGTACTATGTGCCATAATTGTAGTTTTAAGCCCCTAGGCTAACATGTCGTCTAAAGCGGTGTCATGGGTTGTACACGGTATTGAGAGTATT CGGAGGCAGACGGGAAACGCTCCTAGGTCGGGCTGAAGTGGGATGACGAATCAGAGACAATATAAGCGTAAATCAAAGTCTGTTTGGGCGTGATTTATGTAACGCTTCGAAAACCTGCCATGGGAACCTAGAGAAGCAACTCTAACCCCAGTCGATTTAACCCGTGGTAGGCAAGTTCATTGGCCGAAGAGCAAGTGTG ATCCAGTCCCAATCTGCTGTGCTGAAAAGTTTGAGCGAGATGACGGGTAAGGGTCGTCTAAGACCATCGGGAATGTACAGAGGCTCGCCCCTAAGCATCAGTCAAAGGTTATGCACCAGTTCATCCCTCGTGAAAGTATAAGCGGCTTTCAAGAGTGCTAAGTGATCACCTGAAAGTGCAGACGGCCGGCTAGCCGCCT ATGATTTCGATATTATAAGAGTCGGATTATATTGGGCAGTTCAGCGCGTGTAGGTTGGGGTTGGATTTGTCCTGCTACGCCATGCTAACACTATTTTTCGGTATTTCCAGAAAAGTACGCGCTCCATACTTCCTATAACATTGCTTCACACTCTTTTGCGCCAAAACTCCAAATAGTCGTACAAGTGAACAGAATTCAT AACAATATCGATCCTCGAATGCTACGTCCTTTCCCACATTTGACCTCCTTCTCCAACCGACGCGCAATTTTCATAAGCGTGCTTGGTTAAATTCCGTCGGCTGATACTACTAGCGGACGGCGAGGTATGGCACAGGGTAGTACCGATTAGGGTATTAGATGGAGACCTTTACTCGTAACCGTCCCACTTTGAGCCCATG CGGCCGAGAACAAAACTGCAAATGATGGCGCGCTCCAGGAAGGAGTTATACACGACCTGTAACCATCAGTGCACAGGTTCAACCAGGCCATTATTGTAGGGTGTCTTCATCGTGATTCTAAATGGCGATCGTTGCGGTTATTCGGTGGTAATGCTTCCGCGGTAGCTTATAGCTGTGCTTCAAACGTAAGTGACCCAAG GTTAGGCTAGATCCGGGAAGATCTAACGAAAAGAAGTCTCGTGACATCGTGGTTTGCGGCTCGCTTCGGTCATTATTAGATGGCAACTCGCTCCTAAAACCTAGGTGCGATGACGACCACGACGACATCTCAGGAACCCCTCCGAATGGCTTGCTAGGGGAGATCGCCTATAAGGTAGCTTCACAAGTCCTCAGTCTCG GTTGATTCATAGGCGTGTCCGCCGACGGTCAGCGATTCACATCATATAGGAAACGATACGGCCGGGCATTCTAACCGGGGTATAAGGTCGCTTCATCGACAATTGGTCGGGTATGTAAGTGATCGCTTTATACTCTCAATCGTCACTTGTGCCTTTTGGGCAATCTAGTATCGCAAAACCTAGCGGACGGCCCAATAAA GGTTGCCCCTCCTTATGACTCCAGCAACATATACGTATAAGCGTGCTGTCGTGCCGTCAAGCTTAATACGCTGAGCGGATCGATCGAAATTAGAACACCACCGTCAGGCATTTTCCGGGTGCTACACGTACTGTTAGGCTAAAACTATTCTATCAGCGTACGTTAGAAGCGCGATTCTAGTAGCAATGCTTGGTAAGCC')

In [ ]:
print( best_motif )

In [ ]:
# Outputting the final resulting motif.
for kmer in best_motif:
  #print('This is also running')
  print( kmer )


# Gibbs sampler and randomized motif finder.

In [ ]:


import random



alphabet = [ "A", "C", "G", "T"]
alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }
motif_score = 0
best_score = 0
overall_best_score = 0
overall_best_motif = []




def FindProfile( motif, k ):

  # Generate a blank profile matrix.
  profile = [ ]

  consensus_string = ""

  consensus_prob = []

  for nuc in alphabet:

    blank_kmer = [ 1 for i in range( k ) ]

    for mot in motif:

      for j in range( k ):

        #print( mot, j )
        if list( mot )[ j ] == nuc:

          blank_kmer[ j ] += 1

    profile.append( blank_kmer )


  for i in range( k ):

    total_freq = profile[ 0 ][ i ] + profile[ 1 ][ i ] + profile[ 2 ][ i ] + profile[ 3 ][ i ]

    for j in range( 4 ):

      profile[ j ][ i ] = profile[ j ][ i ] / total_freq

      consensus_prob.append( profile[ j ][ i ] )

    consensus_string += list(alpha_dict.keys())[consensus_prob.index( max( consensus_prob ) )]
    consensus_prob = []

  return profile, consensus_string



def Score_Motif( motifs, consensus_string ):

  score_motif = 0

  for kmer in motifs:

    for i in range( len( kmer ) ):

      if kmer[i] != consensus_string[i]:

        score_motif += 1


  #print( score_motif )
  #print( len( motifs ) )

  return score_motif    


def Gibbs( k, N, n, DNA ):

  alphabet = [ "A", "C", "G", "T"]
  alpha_dict = { "A": 0, "C": 1, "G": 2, "T": 3 }
  motif_score = 0
  best_score = 0
  overall_best_score = 0
  best_motif = []
  overall_best_motif = []
  motif = []

  DNA = DNA.split()

  genome_length = len( DNA )
  first_motif = True
  first_run = True

  for j in range( n ):

    motif = []

    # Randomly select a motif.
    for i in range( genome_length ):

      string = DNA[i]
      
      rand_num = random.randint( 0, len(string) - k )

      motif.append(  string[ rand_num:(rand_num + k) ] )

    # Running through N times.
    for run in range( N ):

      # Selecting the string to optimise.
      j = random.randint( 0, genome_length - 1 )

      motif_sans_j = motif[ :j ] + motif[ (j + 1): ]

      profile, consensus_string = FindProfile( motif_sans_j, k )

      freq_table = {}

      # Finding the profile most-probable kmer within the jth string.
      for m in range( 0, len( DNA[ j ] ) - k + 1 ):

        kmer = DNA[ j ] [ m:(m + k) ]
        #print( kmer )

        prob = 1

        for p in range( k ):

          # There may be an error here.
          prob *= profile[ alpha_dict[ kmer[ p ] ]  ] [ p ]

        freq_table[ kmer ] = 1 - prob

      #motif[ l ] = min( freq_table, key=freq_table.get )
      motif_sans_j.insert( j, min( freq_table, key=freq_table.get ) )

      motif = motif_sans_j.copy()

      motif_score = Score_Motif( motif, consensus_string )


      if first_run == True:

        overall_best_score = motif_score
        first_run = False



      if first_motif == True:
        best_score = motif_score
        best_motif = motif.copy()
        first_motif = False

      elif motif_score < best_score:

        best_motif = motif.copy()
        best_score = motif_score

      #print( motif_score, best_score, overall_best_score)

    if first_run == True:

      overall_best_score = best_score
      first_run = False

    if best_score < overall_best_score:

      overall_best_motif = best_score
      overall_best_motif = best_motif.copy()

    #print( overall_best_score, motif_score )

      
  # Outputting the final resulting motif.
  blank = ""

  for kmer in overall_best_motif:
    #print('This is also running')
    #print( kmer )

    blank += " " + kmer

  print(blank)



  return overall_best_motif

    





In [ ]:
best_motif = Gibbs( 15, 2000, 50, "ACCTACGACTAAATAGTACCTCCTACGTGAGCCCGCGGCGCACCCCGATAAATACCGTCGGCTTGGAGACTCGGGCCCAATGCTGGTTAGATCTACAGAGCGGAACACTGCACCATTACATCGCCCGCATACGCTCTACTGAGGCACCACCACGTGATGCACACACGGACGCGTCGCTAGAGTTTTGCAAACACGAGGAACGTAAAGAGAGAGTGAGGACTTCCCGGTATCGAGAGACGCGGATCTGATAGTCACGACATATCAGTGCAAATTGGTGCCTGGAGCGCATTTACTGCTGTCCGTAGTACACATAGGACCTACGACTAAATA GTACCTCCTACGTGAGCCCGCGGCGCACCCCGATAAATACCGTCGGCTTGGAGACTCGGGCCCAATGCTGGTTAGATCTACAGAGCGGAACACTGCACCACTTAATTGCTGAACATTACATCGCCCGCATACGCTCTACTGAGGCACCACCACGTGATGCACACACGGACGCGTCGCTAGAGTTTTGCAAACACGAGGAACGTAAAGAGAGAGTGAGGACTTCCCGGTATCGAGAGACGCGGATCTGATAGTCACGACATATCAGTGCAAATTGGTGCCTGGAGCGCATTTACTGCTGTCCGTAGTACACATAGGACCTACGACTAAATA CCTTTAATAGGACCTGGAGGCTTGACCCAGAATCTAGGATTTTGCTACTCCGAAAACCATCCTACCCGTGCAACCGCACGGGACTACGGATCGGCGACGAACCAATTCGAACCGGAAAGTATGGTAGGTCTCCTCTACCGCGATTAAATTCTCTCAGTCGCGCTGCTGATCAAGACACACCCACGTGCTGAATAGTCAGGCGTCCAAGGTCGTGAAGCCATTAGGACTTCGTGTGTCCGGAGCTCGGCAAGCTCACGTGTTTTTGCGGGCGTTCCGCCTTCTACCACTCTCCAAAGTTTCTTCTTTTCCTTTGAAGTTTGTCATACAGCG GGAACCTACGTACCCGGTTATTTAATAGTCGACCCGGAGAGGGGTTTAATTGCACCAACAAACTACGTTATGGCTAATTCGGGAGACCTATACACCCTATTGTGAAAGAATCCTGTACCAAGTTGCGTTGTTCAAGCTGGAGGAGCTAACCTCTGACCGCATCGTTCGAAGTCAGCTGCCGATTATCTTTAAGGAGAGAGTGGGGGCGTTCGCCCCCCGGGTTAAAAATGCGATCATTTACCCTAGCAATCCGGTATTGAGCGGCAGGTGACCTACTTTAGAGGCGGAGCTCATAATTACACTGACATCCAACTTGGTAGAATGGGCTCC CATAAATAAACGATGCGGGGCCTGATTAGGCCTAGGTGACGCACACTATTTCATGCATGCCTTATATCGTTAATTATGAGATGCTTTGTAAATGATGTACAACACCGACCAGGGCCCTCTTGTAAAGATGAACAAGATCGGAAGGCGACCAAAATCGCAATAAGCACCGCCTTGCTGAATGAAAGATGCCAAGAGCTTGCCCAAACAGTGTTCCGCCTGCGATGTTGGGGGTTAATCAGATAGCGAGCCCAAGCATTCTAAACCGAATAAAGACAGAGGCTTTACTTGCATAGACTAAGAGTAAAGGTTGATACATACGTCGTTGAAAGG GTCCTTAGTCCGCCTTAGCCCCCTCAAGGGAATAATACAAGTCCCACCAATCATGCGCTTGCCCCATTCCCCATGAGGGGATTGGATCAATCTCCCGGAATAGAGAAAGTTCCTCAAGGTATTGCTGAACATCGTCCTTAATAGTCACGTGCGTCAGTGGGGAAGGGGCTCATACCGGCCCAATGGTAGAAGCCAGTCGTTTTTGAGCGGAGGGATCCGTCTTCGCAACTCTCCAATGGCTAAAGCCCGTAACACAAGCCCGGCTGGCCCCTTGCGCCTTAGTTCAGTGCCCGGGGAGCAGGTGTCTCTGTACGGCTGCACTCTCGGACG TTGCTGCACATCTTGGCGGCGTTTAGCCCGACGACCGCCACCCGTGGACTGATATCCTTACGCCGTATGTCAGTGACTGTTGAGTCAGGTCTTTACAGGAGGCCGTAAGTTGAAGATTTAAGGCTCTTCGCTAACTGTCCGTAATTCGAGTGCAAGCCATAAGGTTTGGGTGGTTGCCCTATTGCTGACTAGACGTGGCATCCAGCAACAGAGCCGCATTATCGACCTGAAACGGGCATTCTCCTCTCAGTTAATCATGCGGACTGCACGTGCGGTAGCCCCGTCGAAAGTTGTCCTTGCGACCAAGTGGATATGGATTTTTATTATTCT CTGTCCTAATTTTTATTGAGTACAGGCATGGCGTTTGATTTCCTTTCCAATGACGGATGTTACGAGCTATGTAAGCTACCCGGCTGCAAGGATGGATGGCAGTCGGTGGTCACTTTACTCGCGTCTGTTCTGAGGCCTTCATATAGCATGTCACGTGACAACCTCTTTACACACGCGAGAGAATGTTTAGGGAATTCCTTCGCATAAATAAACTGCCCTGTCCCTTTAGTTGAGCTTATGTGATGCTGGCACTGACGGGAGAAGTGCGTCCATTTACTCATGGCCCACCCTATTGCGCTATTAATGGTTCAATAGTACATTTGTAAGGTT AAGCGATATATGAACGAGAGCTTGCGCGACAAGTCGGGTCCGGCCTTTGGGGGAGTTAGCGAGCGGCACCCTACGCCTGAATTTTGAGGCCTCGCATACAACATTTTGAACCCTCGGACGTGTCGGGCTCAGATCCACAAAGAGGGATAGCATTGTGAGGTTAGTAAGGCTGTTCGACTAGACATTATGACCGTTGGAACTGTTTTCGGAGTGCTGCAATCCCGGATCATACAGAGCTAACGGGCTCAGGGCATATTGGTGAAATCCCGCGCGGGCCCCTAAAGGCTTGCTGCACCGAAGACCGAGAGGGACCTTGGCTTCCCTCCGCTG CGCAGTCTGATGTCTGACGCGTGCTGGTCGGGGTGCCCAAAAAGCCGTCTCACTTGCTAGAGGTAATGGCCACCCTATCATTGAACCCTATGTCTGTCGTGAAAGGGACTAACATATACAAATGTAGTCGTTAAAGTCTCAGCCTCATACAAGTCTCCGAGGAGATGCTGTCATGTCCTACTATGCGTGCGCGCTTGAGTTCCATCCCGGGTCATTCTTGCGGGTGTGAGAGTAGAATTTGGAGTACGGGTCGGCTCAAACTGGTTATGCATAAATCGTTTGTTTTGTCATTTCTTGGAAGAAACGCAACTTCACGCAAGTGGGTGTAAT CCCTCCTGGACCAACAGCGAGTGAGCGTGTTAGTACGTCGGTGGGGAGAGCTTAGAGCCGGTCTGTATACCTTCATGCATTGTGCTTCGCTAAAGCATTTATAACAACGACCTTCGCACTTCCGCCGCTCCTTGGATAGCTAAAACAGGTCCGACCATCGCCAAGCTGTAGGCTCAAAGCTACAATACTTGAGCTTCCCTCGAATCTCCGTCCATTCCTTATAAAGACCTAGCGTACTCTGGTGAGTTACCGAGGAAAGGATTAACCAAACTTTGATATGACACACCCTTGGGCTGAATAACGCCCGGCGGTTTAGAATAAATAAGAAAT CCCTTCCTTAGGATCATCGCAATTCCGCCTGCACACCCTCCATAGAAGCCGAAAGCGCCAAGTTAAACAGCAGTTGCTACACGCTAGGCACAGTAACATCATAGATTTGAAATTCCTGGCGAGGTAAGGACTAGGTTAGGCACCAACCCTTAAGGTAGTGAAACGGAGAGAGTAGCCCGTCCATCACCATTGTTACGAAGGCAAACCTAGTCTAGGTTGTCTGTACCTGACCCCTGCCTTGCGCTAGCCCCGGGGGCAGCTATATCGTTTTTAACGACTGTCGATAGAGGCAAAATCAGCACTTCATTGCTGAAAACCTCGAAGGCAGCT GCAGCACGCACCCTATTGCTATTCTTCTCAGCATGGCTAAAGTGCTCGACCTATTCAATAAAACCCATATACGGATAAGATCCTGCGAGAGCAGGCCTTCCAACTAGAGGTGGTGTTTTATACACAGTTCCACCAGGGAAACGGGGGACAAGAGATACCATACATAGCTTTCGTTATCGAGTTTGACGGAAAACCACCCATTGTACATAGTATGACTGTTGTTCACTAAGGCTCCGAATATCGGATAACGATGGTTATCCGTTGTCACCGACCGCTTCGACCACCCGGCCCGAGTTTATGTATCTCCATCAGAGGGTACATACACATGCA GCATATCAATTGAGGGGCTCTTCGAAATCGCACACGGAGGATCTAACGGCCAGGCGTGTTTGTGTATTGCTGAGGTGGTCCACGTAATAAGTAAAAGATTCTCAAGGGGGACATGCGTAACCGAGGTTACCCCTTTAGGGGCCGATACTTGGCCGGCCCTAAGATATAGAGCTGTTTCCCTTAGCGTACATGCCATCCGTTAGTATTACTTACTGTATGCTTACCCTATTGCTGTCAAGTTCGTTTGAGTTCACAGGCCCTCTTTTGACAAGAATCAGAGGATTGTCCAACATGAAGCACCCGACGTCAGGTAAGCTGGTAGCAGTAAAT GCCGACGACTGCATGTTAGAATGTCTAACCTGACGACCAGCGACCGGGCCTCTCGTGCCTGTAGCCATGGGGGCCTTGAATATGGAGTCAGACAGCCCGCATGGTCGGATTCTCCTAGGGATGGGTCATCGAGGGTACCAATGTAAAGGATCCTGAAATGTTGGAAAACGCACCCTATTCGGGAAGTGACTCCCAGAAGGTAGCCGCAAACTACTACGATCTCCTTTACCGCCCACCCAGGATCCATATCACCTAGCTGAGGACGTGGGGCGAGACAGTATCAGGGTTAACTGATTCTCACGTTCGGTAGTGATATTAAGATATTTCACG CTCGTGCAAATGGCTAGTGAGTTTAGCTTCGGCGAAAGTGCACCGGAACTCCTTGGCCCCGATGTTTATCGAACCGCTATTAAGCACGACGACCAGACCCTTGTATCTGCGGCAGTTCCTCAAAAACCTCGATGACCAAATGTCACCCTCGAGCTGAATGACGTAGCTCCGGTCGTCACGACTCCGGGGATTTCGTTGTTTGCACGAGAATACGTTGTTATTAAGTAGCTGTGAACTCTGCCAACACAGCCGAAGTTTATATGACGGAGAGGGGACGCCTCTAGCGTTACTATGGCCCATTTAAGGGATATCGGACGTGTAGTCTCACCG CGGTACAGTGACGCTCCTATTGCTGAAATTCGATGCGCCGCACGCATATCCCAGCCAAAGCGGTTGCTACATGAATTGCCCCGATATCTAGGAACGATGCGCTTCAAGTCGCGACTTTAGCTAAATCATAGATCCACGGAGGTTATAGACAGTTTTCACCGCCAAAGATTTTTATGTTACACCGCATTGGCCTGATCAATTGCCCTACCTGCAAATCCTCCTACGTACATTGTACAAATGCATCGGGTGGGCAGGCAATGCGACAACGACGAAGTAGGCTTAGTCATGATGATCCGGTAAGACATTTACTACAGGAACAAACAAGTGCGA TATGGATCGTGAGCGACGTTCTGTCCGAGTGCAGTGACGCCGCACTCTCCTGGACGCAGCAGCTGGCTCACCCATATGCTGAAGACTAGCGCGGTATCTCGTGGTAAGTTATCTATCATGCACGGTGGGTCTGCAATACTTCCGCCCACTATGGCGTGTTGAAGGGTTAGGTACGTGGACTCGGGTGGGATCCACACCGCCTCTCCATTCTAGGGTGCACCAGCTCGTTACCGGCTGCTCCAGTGTGGGGACTCCCGTCCGATCAATAAGCCCAATAGAAGAAGGTTGATTGCGTGGTCTGATACCCCACGTAAACACAACGCTGCCTTC CTACGCTGGAGTCGTAGAGGCCTTATGCTCTCAGAAGGCGGGTTATGGATAGTCGCACCACTTACACATAGGCTACCGAACGAGGTTCACTGACGGGTACTGGCTTCTGCGTGCCGGTACGCCTATCGCGGGCTCTGCTCAGAAGGAATGGTGTCGTCGAGTTTATTCAAAGCGTTGAATGCAGTTGCACTTCCCAATCGAACCCAAAAAGCGACGAAGGACAGTTTAACCCAGGGTTGGTGTCCATAACTCTGCGACGATCCTCACGGTGAGCGGTATCGGAGTCACCGATTTGCTGAATGAGATTAAGTTTGGGACTTTGCTTCGCCT CGGGGTAGCGCTGATCTGGTTAACTACTAACCCCGAAAGCTCAGCCTAGGTCTATCTCATGTAATCGTTGACGTTTCTCTGTAGCACAGTGCCCCGATAGCTAAACTTGCTATTGCTGAAGTTGTTATAGATTATCGGTGTGACGGGTGAAAGATTGAGAGGGTCGCATTGTAAAGCATCATTCTGATACATACACGAGATTCCGGTTACACGAATACCCTTTTCACACCTTGACAAGTCCTATGCCGCGTGCCCTAGCCCTCTGGTGATCGGTCACGCACAGTGACAAGCCCAACATACCATATAAGTAACGCCCTAGTTTTATTAAGT" )

In [ ]:
print( best_motif )

# Finding the Hamming distance between a gene and multiple strings of DNA.

In [ ]:
DNA = "CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG"

DNA = DNA.split()

def HammingDistanceArray( fg, sg ):


  distance = 0

  for i in range( len( fg ) ):

    for j in range( len(sg) ):

      
      if list( fg[ i ] ) [ j ] != sg[ j ]:
        distance += 1

  return distance


HammingDistanceArray( DNA, "TAGTTTC" )

